# 11 - Testando Hipóteses

### 11.1 - Voltar ao problema do euro

- Problema apresentado no capítulo IV. 
- Uma moeda belga foi lançada 250 vezes, os resultados foram, 140 caras e 110 coroas;
    - Mas esses dados evidenciam que a moeda é tendenciosa e não justa?
    
- Os dados sejam a favor de uma hipótese se o fator de Bayes for maior que 1.

- Duas hipóteses a serem consideradas:
    - F para a hipótese de que a moeda é justa;
    - B para a hipótese de que é tendenciosa.

In [20]:
import thinkbayes

class Euro(thinkbayes.Suite):
    def Likelihood(self, data, hypo):
        x = hypo / 100.0
        heads, tails = data
        like = x**heads * (1-x)**tails
        return like

In [21]:
euro = Euro()
data = (140, 110)
hypo = 50
likeF = euro.Likelihood(data, hypo)
print(likeF)

5.527147875260445e-76


- Probabilidade muito pequena;
- Necessárias duas probabilidade para fazer uma relação;
- Não é fácil calcular a verossimilhança de B;
- Examinar os dados antes de definirmos a hipótese;
    - Definiu como tendêncioso a probabilidade de cara sendo 140/250;

In [22]:
actual_percent = 100.0 * 140 / 250
likeB = euro.Likelihood(data, actual_percent)
print(like)

3.358289985239045e-75


In [23]:
likeB/likeF

6.075990838368512

- Usar os dados para definir uma hipótese, torna essa hipótese fictícia;
    - Qualquer conjunto de dados seria uma evidência a favor de B;

### 11.2 Fazendo uma comparação justa

- Definir B sem examinar os dados;
- A forma da moeda tem algum efeito em x;
- Eu acho que a moeda é tendenciosa, de modo que x seja 0.6 ou 0.4, mas não tenho certeza de qual;

In [24]:
like40 = euro.Likelihood(data, 40)
like60 = euro.Likelihood(data, 60)
likelihood = 0.5 * like40 + 0.5 * like60
likelihood

7.357772729537146e-76

Fator de Bayes

In [25]:
fb = likelihood/likeF
fb

1.3312060570100888

- Não tem idéia do valor de x;
- b_uniform, para representar sub-hipóteses de 0 a 100.

In [26]:
b_uniform = Euro(range(0, 101))
b_uniform.Remove(50)
b_uniform.Normalize()

0.9900990099009908

In [27]:
def SuiteLikelihood(suite, data):
    total = 0
    for hypo, prob in suite.Items():
        like = suite.Likelihood(data, hypo)
        total += prob * like
    return total

In [28]:
like = SuiteLikelihood(b_uniform, data)

In [29]:
like

2.5796490229198124e-76

In [30]:
# Fator Bayes
like/likeF

0.4667233591607601

- Os dados são uma evidencia fraca de b_uniform, em comparação com F.

In [31]:
class Euro(thinkbayes.Suite):
    def Likelihood(self, data, hypo):
        x = hypo / 100.0
        heads, tails = data
        like = x**heads * (1-x)**tails
        return like
    
    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        return self.Normalize()
    
    def Normalize(self):
        total = self.Total()
        
        factor = 1.0 / total
        for x in self.d:
            self.d[x] *= factor

        return total

In [32]:
b_uniform = Euro(range(0, 101))
b_uniform.Remove(50)
b_uniform.Normalize()
likelihood = b_uniform.Update(data)
likelihood

2.5796490229198124e-76

In [34]:
#Fator bayes
fb = likelihood/likeF
fb

0.4667233591607601

### 11.3 - A priori triangular

- Maior probabilidade a valores de x próximos a 50%;

In [35]:
def TrianglePrior():
    """Makes a Suite with a triangular prior."""
    suite = Euro()
    for x in range(0, 51):
        suite.Set(x, x)
    for x in range(51, 101):
        suite.Set(x, 100-x) 
    suite.Normalize()
    return suite

In [36]:
b_triangle = TrianglePrior()
likelihood = b_triangle.Update(data)
likelihood

4.6354035014189224e-76

In [37]:
#Fator Bayes
likelihood/likeF

0.838661024823856

- Os dados são evidência fraca contra B.

Dependendo da definição escolhida, os dados podem fornecer evidências a favor ou contra a hipótese de que a moeda é tendenciosa, mas em ambos os casos é uma evidência relativamente fraca.